In [67]:
import pickle
path = "data/Physics/0_uncopynode.pkl"
with open(path, 'rb') as f:
    data = pickle.load(f)

In [1]:
# print(data.edge_index.shape, data.train_mask.shape)
# s = set()
# for e in data.edge_index[1]:
#     s.add(e.item())
# print(len(s))
# data.edge_index
# data.train_mask

In [55]:
class DotDict(dict):
    def __init__(self, **kwds):
        self.update(kwds)
        self.__dict__ = self

path_sbm_train = "data/SBM/SBM_CLUSTER_train.pkl"
path_sbm_val = "data/SBM/SBM_CLUSTER_val.pkl"
path_sbm_test = "data/SBM/SBM_CLUSTER_test.pkl"
with open(path_sbm_train, 'rb') as f:
    sbm_train = pickle.load(f)
with open(path_sbm_val, 'rb') as f:
    sbm_val = pickle.load(f)
with open(path_sbm_test, 'rb') as f:
    sbm_test = pickle.load(f)
# sbm_train

In [69]:
# for tmp in sbm_train:
#     print(len(tmp.node_feat))
sbm_train[0].node_feat
# edge_index = torch.tensor([[],[]])
# print(edge_index)
# print(torch.cat((edge_index, torch.nonzero(sbm_train[0].W).T+100), 1))
# x = torch.BoolTensor([0]*10)
# x[1:3] = True
# print(x)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0,
        0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 5, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0],
       dtype=torch.int16)

In [83]:
from torch_geometric.data import Data
K=100
NBTRAIN = len(sbm_train)
NBVAL = len(sbm_val)
NBTEST = len(sbm_test)

import torch
for k in range(K):
    idx_base=0
    edge_index = torch.LongTensor([[],[]])
    x = torch.tensor([])
    y = torch.tensor([])
    for idx_graph in range(k*int(NBTRAIN/K),(k+1)*int(NBTRAIN/K)):
        sub_graph = sbm_train[idx_graph]
        edge_index = torch.cat(
            (edge_index, torch.nonzero(sbm_train[0].W).T+idx_base), 1)
        x = torch.cat((x, torch.unsqueeze(sub_graph.node_feat, 1)), 0)
        y = torch.cat((y, sub_graph.node_label), 0)
        idx_base += len(sub_graph.node_feat)
    idx_train = idx_base
    for idx_graph in range(k*int(NBVAL/K), (k+1)*int(NBVAL/K)):
        sub_graph = sbm_val[idx_graph]
        edge_index = torch.cat(
            (edge_index, torch.nonzero(sbm_train[0].W).T+idx_base), 1)
        x = torch.cat((x, torch.unsqueeze(sub_graph.node_feat, 1)), 0)
        y = torch.cat((y, sub_graph.node_label), 0)
        idx_base += len(sub_graph.node_feat)
    idx_val = idx_base
    for idx_graph in range(k*int(NBTEST/K), (k+1)*int(NBTEST/K)):
        sub_graph = sbm_test[idx_graph]
        edge_index = torch.cat(
            (edge_index, torch.nonzero(sbm_train[0].W).T+idx_base), 1)
        x = torch.cat((x, torch.unsqueeze(sub_graph.node_feat, 1)), 0)
        y = torch.cat((y, sub_graph.node_label), 0)
        idx_base += len(sub_graph.node_feat)
    idx_test = idx_base

    x = torch.zeros(x.shape[0], 7).scatter(1, torch.tensor(x,dtype=torch.int64), 1)[:, 1:]
    train_mask = torch.BoolTensor([0]*idx_base)
    train_mask[0:idx_train] = True
    val_mask = torch.BoolTensor([0]*idx_base)
    val_mask[idx_train:idx_val] = True
    test_mask = torch.BoolTensor([0]*idx_base)
    test_mask[idx_val:idx_test] = True
    with open("data/SBM/{}_uncopynode.pkl".format(k), "wb") as f:
        pickle.dump(Data(edge_index=edge_index, x = x, y = y, train_mask = train_mask, val_mask = val_mask, test_mask = val_mask), f)
    break

/tmp/ipykernel_562791/3679550560.py:38: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.zeros(x.shape[0], 7).scatter(1, torch.tensor(x,dtype=torch.int64), 1)[:, 1:]
